In [1]:
import os
import json

path = r"G:\My Drive\my_work\temp\instances_train2017.json\instances_train2017.json"


In [2]:
# open json file
with open(path, 'r') as f:
    data = json.load(f)


In [3]:
data.keys()

dict_keys(['info', 'licenses', 'images', 'annotations', 'categories'])

In [4]:
data['annotations'][1]

{'segmentation': [[247.71,
   354.7,
   253.49,
   346.99,
   276.63,
   337.35,
   312.29,
   333.49,
   364.34,
   331.57,
   354.7,
   327.71,
   369.16,
   325.78,
   376.87,
   333.49,
   383.61,
   330.6,
   379.76,
   321.93,
   365.3,
   320.0,
   356.63,
   317.11,
   266.02,
   331.57,
   260.24,
   334.46,
   260.24,
   337.35,
   242.89,
   338.31,
   234.22,
   338.31,
   234.22,
   348.92,
   239.04,
   353.73,
   248.67,
   355.66,
   252.53,
   353.73]],
 'area': 1545.4213000000007,
 'iscrowd': 0,
 'image_id': 200365,
 'bbox': [234.22, 317.11, 149.39, 38.55],
 'category_id': 58,
 'id': 509}

In [8]:
(data['images'][0])

{'license': 3,
 'file_name': '000000391895.jpg',
 'coco_url': 'http://images.cocodataset.org/train2017/000000391895.jpg',
 'height': 360,
 'width': 640,
 'date_captured': '2013-11-14 11:18:45',
 'flickr_url': 'http://farm9.staticflickr.com/8186/8119368305_4e622c8349_z.jpg',
 'id': 391895}

In [9]:
(data['annotations'][0])

{'segmentation': [[239.97,
   260.24,
   222.04,
   270.49,
   199.84,
   253.41,
   213.5,
   227.79,
   259.62,
   200.46,
   274.13,
   202.17,
   277.55,
   210.71,
   249.37,
   253.41,
   237.41,
   264.51,
   242.54,
   261.95,
   228.87,
   271.34]],
 'area': 2765.1486500000005,
 'iscrowd': 0,
 'image_id': 558840,
 'bbox': [199.84, 200.46, 77.71, 70.88],
 'category_id': 58,
 'id': 156}